# Horses or Humans Classification

---
```
Step 0. Library Import
Step 1. Dataset Load
Step 2. Data Preprocessing
Step 3. Modeling
Step 4. Model Compile
Step 5. Model Checkpoint
Step 6. Model Fit
Step 7. Model Evaluate & Svae
Step 8. Reload Model
```

## Step 0. Library Import

In [27]:
import urllib.request
import zipfile
import numpy as np
from IPython.display import Image

import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

## Step 1. Dataset Load

In [7]:
train_url = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
valid_url = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"

In [8]:
train_zipfile_path = './data/train_horses_or_humans.zip'
valid_zipfile_path = './data/valid_horses_or_hamans.zip'

In [10]:
# train set unzip
urllib.request.urlretrieve(train_url, train_zipfile_path)
zip_ref = zipfile.ZipFile(train_zipfile_path,'r')
zip_ref.extractall('./data/train_horses_or_humans/')
zip_ref.close()

# test set unzip
urllib.request.urlretrieve(valid_url, valid_zipfile_path)
zip_ref2 = zipfile.ZipFile(valid_zipfile_path,'r')
zip_ref2.extractall('./data/valid_horses_or_hamans/')
zip_ref2.close()

## Step 2. Data Preprocessing

In [11]:
train_data_path = './data/train_horses_or_humans/'
valid_data_path = './data/valid_horses_or_humans/'

In [24]:
train_idg = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 5,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'reflect',
)

In [25]:
valid_idg = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 5,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'reflect',
)

In [22]:
train_gen = train_idg.flow_from_directory(
    train_data_path,
    target_size=(300,300),
    batch_size=64,
    class_mode='categorical'
)

Found 1027 images belonging to 2 classes.


In [23]:
valid_gen = valid_idg.flow_from_directory(
    train_data_path,
    target_size=(300,300),
    batch_size=64,
    class_mode='categorical'
)

Found 1027 images belonging to 2 classes.


## Step 3. Modeling

In [28]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 64)        3

## Step 4. Model Compile

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

## Step 5. Model Checkpoint

In [35]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

## Step 6. Model Fit

In [36]:
model.fit(
    train_gen,
    validation_data=(valid_gen),
    epochs=12,
    callbacks=[cp]
)

Epoch 1/12
17/17 [==============================] - ETA: 0s - loss: 0.1256 - acc: 0.9484
Epoch 00001: val_loss improved from inf to 0.09423, saving model to my_checkpoint.ckpt
17/17 [==============================] - 56s 3s/step - loss: 0.1256 - acc: 0.9484 - val_loss: 0.0942 - val_acc: 0.9630
Epoch 2/12
17/17 [==============================] - ETA: 0s - loss: 0.1042 - acc: 0.9601
Epoch 00002: val_loss improved from 0.09423 to 0.07853, saving model to my_checkpoint.ckpt
17/17 [==============================] - 54s 3s/step - loss: 0.1042 - acc: 0.9601 - val_loss: 0.0785 - val_acc: 0.9786
Epoch 3/12
17/17 [==============================] - ETA: 0s - loss: 0.0983 - acc: 0.9679
Epoch 00003: val_loss improved from 0.07853 to 0.07292, saving model to my_checkpoint.ckpt
17/17 [==============================] - 54s 3s/step - loss: 0.0983 - acc: 0.9679 - val_loss: 0.0729 - val_acc: 0.9727
Epoch 4/12
17/17 [==============================] - ETA: 0s - loss: 0.0890 - acc: 0.9649
Epoch 00004: val_l

## Step 7. Model Evaluate & Save

In [37]:
model.load_weights(filepath)

In [38]:
model.evaluate(valid_gen)

17/17 [==============================] - 21s 1s/step - loss: 0.0169 - acc: 0.9942


[0.01685025915503502, 0.9941577315330505]

In [40]:
model.save('./model/hoh_cnn_0181.h5')

## Step 8. Reload Model

In [41]:
mymodel = tf.keras.models.load_model('./model/hoh_cnn_0181.h5')

In [42]:
mymodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 64)        3